In [1]:
import velocity_metrics.utils.constant as const 
import velocity_metrics.eulerian.eulerian_drifters as eulerian  
import datetime
import sys   
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import cartopy 
import warnings 
warnings.filterwarnings("ignore") 
sys.path.append('/Odyssey/private/t22picar/2024_DC_WOC-ESA/')
import json

import argparse
import numpy
import sys
import os
import scipy.interpolate as interpolate
import datetime
import pickle
import logging
from typing import Tuple, Optional
from scipy.ndimage import gaussian_filter
from velocity_metrics.reader import read_utils_xr
from velocity_metrics.utils import load_parameters
from velocity_metrics.reader import read_drifters
import velocity_metrics.utils.constant as const

from velocity_metrics.eulerian.eulerian_drifters import compute_error

logger = logging.getLogger()
handler = logging.StreamHandler()
logger.addHandler(handler)


In [3]:

xp_name = "unet_uv_drifters_aoml_15m_10y_11d_bathy_no_sst_mae_SB"
print(xp_name)
base_output="/Odyssey/private/t22picar/multivar_drifter/"
path_files=f'{base_output}rec/{xp_name}/daily/'

input_dict = f'{base_output}metric/dictionary/'
input_drifter = '/Odyssey/private/t22picar/2024_DC_WOC-ESA/dc_data/drifter/'
outputdir = f'{base_output}rec/{xp_name}/metric'

depth=15
depth_formatted = "{:02}".format(depth)

unet_uv_drifters_aoml_15m_10y_11d_bathy_no_sst_mae_SB


In [4]:
drifter_paths=[input_drifter+f'Drifters_AOML_region_GulfStream_{depth_formatted}m_20190101T000000Z_20200101T000000Z.pyo.gz']
 
data_type_file = input_dict+f'{xp_name}.json' 

first_date = datetime.datetime.strptime('20190101T000000Z', const.FMT)
last_date  = datetime.datetime.strptime('20191231T000000Z', const.FMT)

region = input_dict+'region_GulfStream.json'
sdepth= 1

dic_drif = read_drifters.load_drifter_pickle(list(drifter_paths))

In [5]:
x = load_parameters.sel_region(region)
lllon, urlon, lllat, urlat, coords, name = x
logger.info(f'Load velocity {data_type_file}')
x = load_parameters.sel_data(data_type_file)
cpath, cpattern, match, dic_name, depth, data_type, label, spatial_cov_h = x
box = [lllon, urlon, lllat, urlat]
_read = read_utils_xr.read_velocity
VEL, coord = _read(cpath, cpattern, match, first_date, last_date,
                    box=box, depth=depth[sdepth], dict_name=dic_name,
                    stationary=False, compute_strain=False,
                    compute_ow=False, compute_rv=False)

out_dic = compute_error(VEL, coord, dic_drif,_av=3600)

mean_dic = {}
std_dic = {}
# Compute statistics
logger.info('Compute statistics')
mean_dic = {}
std_dic = {}

for comp in out_dic.keys():
    _tmp = numpy.array(out_dic[comp])
    
    mean_dic[comp] = numpy.sqrt(numpy.nanmean(_tmp**2))
    std_dic[comp] = numpy.nanstd(_tmp)
for comp in ('Eastward', 'Northward', 'Norm'):
    x = numpy.array(out_dic[f'{comp} speed']).ravel()
    y = numpy.array(out_dic[f'{comp} drifter speed']).ravel()

    mse = numpy.nanmean(numpy.array(out_dic[f'{comp} difference'])**2)
    #mean_dic[f'{comp} Quadratic Error (%)'] = 100 * (mean_dic[f'{comp} difference']
    #                                        / mean_dic[f'{comp} drifter speed'])

    mask = (~numpy.isnan(x)) & (~numpy.isnan(y))
    x_nm = x[mask]
    y_nm = y[mask]
    if len(x_nm) > 1:
        #mean_dic[f'{comp} Correlation'] = numpy.corrcoef(x_nm, y_nm)[0, 1]
        cov = numpy.cov(x_nm, y_nm, ddof=1)[0][1]
        var = numpy.var(y_nm, ddof=1)
        print(comp)
        print(mse)
        print(var)
        mean_dic[f'{comp} Explained Variance (%)'] = (100 * cov / var)
        mean_dic[f'{comp} Ronan Explained Variance (%)'] = (1 - (mse / var))*100


None
Percent: [##############################] 99.73%, , Eastward
0.05082217615202712
0.1245419083749128
Northward
0.056542693823162044
0.12765019693971827
Norm
0.06692720039089566
0.09462998312091997


In [6]:
#unet_uv_drifters_aoml_15m_10y_11d_bathy_no_sst_mae_SB
print("Lucile Explained Variance (%)")
print(mean_dic['Eastward Explained Variance (%)'])
print(mean_dic['Northward Explained Variance (%)'])
print(mean_dic['Norm Explained Variance (%)'])

print("Ronan Explained Variance (%)")
print(mean_dic['Eastward Ronan Explained Variance (%)'])
print(mean_dic['Northward Ronan Explained Variance (%)'])
print(mean_dic['Norm Ronan Explained Variance (%)'])

Lucile Explained Variance (%)
62.01037944697702
59.04939009334854
60.54946593038364
Ronan Explained Variance (%)
59.19271126066627
55.704969378258106
29.274846952709666


In [46]:
#globcurrent_15m_4th
print("Lucile Explained Variance (%)")
print(mean_dic['Eastward Explained Variance (%)'])
print(mean_dic['Northward Explained Variance (%)'])
print(mean_dic['Norm Explained Variance (%)'])

print("Ronan Explained Variance (%)")
print(mean_dic['Eastward Ronan Explained Variance (%)'])
print(mean_dic['Northward Ronan Explained Variance (%)'])
print(mean_dic['Norm Ronan Explained Variance (%)'])

Lucile Explained Variance (%)
61.56910421092435
58.33525278480492
57.078011233391116
Ronan Explained Variance (%)
53.901694002718855
51.66362840680592
27.815758754339946


In [32]:
#unet_uv_drifters_aoml_15m_10y_11d_bathy_no_sst
print("Lucile Explained Variance (%)")
print(mean_dic['Eastward Explained Variance (%)'])
print(mean_dic['Northward Explained Variance (%)'])
print(mean_dic['Norm Explained Variance (%)'])

print("Ronan Explained Variance (%)")
print(mean_dic['Eastward Ronan Explained Variance (%)'])
print(mean_dic['Northward Ronan Explained Variance (%)'])
print(mean_dic['Norm Ronan Explained Variance (%)'])

Lucile Explained Variance (%)
66.78030066296844
61.41593866548496
60.870011520591106
Ronan Explained Variance (%)
56.53535999046048
55.71714358961486
32.31027043608532


In [28]:
#Neurost
print("Lucile Explained Variance (%)")
print(mean_dic['Eastward Explained Variance (%)'])
print(mean_dic['Northward Explained Variance (%)'])
print(mean_dic['Norm Explained Variance (%)'])

print("Ronan Explained Variance (%)")
print(mean_dic['Eastward Ronan Explained Variance (%)'])
print(mean_dic['Northward Ronan Explained Variance (%)'])
print(mean_dic['Norm Ronan Explained Variance (%)'])

Lucile Explained Variance (%)
65.37024749831846
58.85968051958663
62.34793046602825
Ronan Explained Variance (%)
60.69946687881321
57.62249982614518
32.201633171030494


In [ ]:
64.769	59.117	61.632

In [ ]:
xp_name = "unet_uv_drifters_aoml_15m_10y_11d_finescale_NA_log_wind_bathy_res"
print(xp_name)
base_output="/Odyssey/private/t22picar/multivar_drifter/"
path_files=f'{base_output}rec/{xp_name}/daily/'

if "_0m" in xp_name:
    depth = 0
elif "_15m" in xp_name:
    depth = 15
else: 
    print("no depth in xp name")
    depth = 15
# Formater depth avec deux chiffres significatifs
depth_formatted = "{:02}".format(depth)

input_dict = f'{base_output}metric/dictionary/'
input_drifter = '/Odyssey/private/t22picar/2024_DC_WOC-ESA/dc_data/drifter/'
outputdir = f'{base_output}rec/{xp_name}/metric'

path_dict_product = input_dict+f'{xp_name}.json' 

first_date = datetime.datetime.strptime('20190101T000000Z', const.FMT)
last_date  = datetime.datetime.strptime('20191231T000000Z', const.FMT) 

outputdir = f'{base_output}rec/{xp_name}/metric/GulfStream/'
path_dict_region = input_dict+'region_GulfStream.json'
#drifter_list = [input_drifter+'Drifters_AOML_region_T1_00m_20190101T000000Z_20200101T000000Z.pyo.gz']
drifter_list = [input_drifter+f'Drifters_AOML_region_GulfStream_{depth_formatted}m_20190101T000000Z_20200101T000000Z.pyo.gz']

eulerian.run(drifter_list, path_dict_product, 
             first_date=first_date, last_date=last_date, 
             region=path_dict_region, sdepth=1, output_dir=outputdir) 

unet_uv_drifters_aoml_15m_10y_11d_finescale_NA_log_wind_bathy_res
None
Percent: [##############################] 99.73%, , 

Compute and save statistics in /Odyssey/private/t22picar/multivar_drifter/rec/unet_uv_drifters_aoml_15m_10y_11d_finescale_NA_log_wind_bathy_res/metric/GulfStream/ directory
